In [ ]:
# Importar bibliotecas
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Ignorar mensajes de advertencia para mejorar la legibilidad
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Establecer el estilo de las gráficas utilizando el estilo 'ggplot'
plt.style.use('ggplot')

# **MNIST**

**MNIST** es un conjunto de 70.000 imágenes pequeñas de dígitos escritos a mano por alumnos de instituto y empleados de la Oficina del Censo de EE.UU. Cada imagen está etiquetada con el dígito al que representa. Este conjunto se ha estudiado tanto que, a menudo, se le llama el ‘Hello, Word’ del *Machine Learning*.

In [ ]:
# Cargar el dataset 'MNIST'
mnist = fetch_openml('mnist_784', version=1, as_frame=False)

In [ ]:
# Separar las features (píxeles) y las labels en el dataset
X, y = mnist['data'], mnist['target']

# Convertir las lables de string a integer
y = y.astype(np.uint8)

In [ ]:
# Imprimir las dimensiones del conjunto features (X) y del conjunto de labels (y)
print('dim(X) =', X.shape)
print('dim(y) =', y.shape)

Hay 70.000 imágenes y cada imagen tiene 784 features. Esto se debe a que cada imagen tiene 28x28 pixeles y cada feature representa simplemente la intensidad de un pixel, desde 0 (blanco) hasta 255 (negro).

In [ ]:
def plot_digits(instances, images_per_row=10, **options) -> None:

    size = 28
    images_per_row = min(len(instances), images_per_row)
    n_rows = (len(instances) - 1) // images_per_row + 1

    n_empty = n_rows * images_per_row - len(instances)
    padded_instances = np.concatenate([instances, np.zeros((n_empty, size * size))], axis=0)

    image_grid = padded_instances.reshape((n_rows, images_per_row, size, size))

    big_image = image_grid.transpose(0, 2, 1, 3).reshape(n_rows * size,
                                                         images_per_row * size)

    plt.imshow(big_image, cmap = mpl.cm.binary, **options)
    plt.axis('off')

In [ ]:
# Crear un gráfico mostrando los primeros 100 dígitos del dataset
plt.figure(figsize=(12, 10))
plot_digits(X[:100], images_per_row=10)
plt.show()

In [ ]:
# Calcular la cantidad de ocurrencias de cada dígito en el dataset 'MNIST'
unique, counts = np.unique(y, return_counts=True)

# Crear un gráfico de barras para visualizar la distribución de los dígitos en el conjunto de datos
plt.figure(figsize=(12, 6))
plt.bar(unique, counts, align='center', alpha=0.7, color='steelblue')
plt.xticks(unique)
plt.title('Distribution of Digits in MNIST Dataset')
plt.xlabel('Digits')
plt.ylabel('Number of Digits')
plt.show()

In [ ]:
# Dividir el conjunto de datos 'MNIST' en train y test
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# Establecer una semilla aleatoria
np.random.seed(42)

# Seleccionar 30,000 muestras aleatorias del conjunto de train
indices_train = np.random.choice(len(X_train), 30000, replace=False)
X_train, y_train = X_train[indices_train], y_train[indices_train]

# Seleccionar 5,000 muestras aleatorias del conjunto de test
indices_test = np.random.choice(len(X_test), 5000, replace=False)
X_test, y_test = X_test[indices_test], y_test[indices_test]

In [ ]:
# Crear un pipeline que incluye escalado y un clasificador SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(class_weight='balanced'))
])

In [ ]:
# Entrenar un modelo básico como baseline usando Cross-Validation
pipeline.named_steps['svc'].set_params(kernel='linear')
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Score:", cv_scores.mean())

In [ ]:
# Entrenar un clasificador SVM con kernel RBF
param_grid_rbf = {
    'svc__kernel': ['rbf'],
    'svc__C': [0.1, 1.0, 10.0],
    'svc__gamma': ['scale', 'auto']
}

svc_rbf = GridSearchCV(
    pipeline, 
    param_grid_rbf, 
    cv=5
)
svc_rbf.fit(X_train, y_train)

In [ ]:
# Resultados de entrenamiento obtenidos para el clasficador SVM con kernel RBF
svcRBF_results = pd.DataFrame(svc_rbf.cv_results_)
svcRBF_results[['param_svc__kernel', 'param_svc__C', 'param_svc__gamma','mean_test_score', 'std_test_score']]\
    .sort_values(by='mean_test_score', ascending=False).reset_index(drop=True).head()

In [ ]:
# Realizar predicciones en el conjunto de test
y_pred = svc_rbf.best_estimator_.predict(X_test)

In [ ]:
# Imprimir la matriz de confusión para evaluar las predicciones del modelo en el conjunto de test
print(confusion_matrix(y_test, y_pred))

# Imprimir un informe de clasificación para evaluar el rendimiento del modelo en el conjunto de test
print(classification_report(y_test, y_pred))

In [ ]:
# Calcular y normalizar la matriz de confusión
cm = confusion_matrix(y_test, y_pred)
row_sums = cm.sum(axis=1, keepdims=True)
norm_conf_mx = cm / row_sums

# Establecer los elementos diagonales a 0 para resaltar los errores
np.fill_diagonal(norm_conf_mx, 0)

# Visualizar la matriz de confusión normalizada con resaltado de errores
plt.figure(figsize=(12, 10))
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.title('Normalized Confusion Matrix with Highlighted Errors')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(np.arange(10))
plt.yticks(np.arange(10))
plt.grid(False)
plt.show()